<a href="https://colab.research.google.com/github/ChristopherLuey/Northwestern-ME-441-Gaussian-Process-Bayesian-Optimization-Tool/blob/main/GP_BO_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gaussian Process Baeysian Optimization Tool

**First, upload a CSV file with experimental data**

Data should be in the following format:

```
Column 1 | Column 2 | ... | Column n
x1       | x2       | ... | y
```
Columns represent design variables, rows represent different sample points. There should be no headers, just data.


Example for 1D case:

```
0.529	0.991
1.899	0
7	    -4.959
3.7	  -2.495
```

---

# Follow these steps to use this code:
```
1. Run this to start
2. Set file_name to the path of the CSV file and run
3. Run this to indicate the inclusive limits of the design space
4. Run this to determine the point of greatest acquisition
5. Run this to graph the result for 1D or 2D cases
6. Run this to determine the mean and covariance at a specific point
```




In [ ]:
# @title 1. GP BO Tool IDEAL Lab { form-width: "100%", display-mode: "form" }
# Run
!pip install gpytorch -q
import matplotlib.pyplot as plt
import numpy as np
import gpytorch
import math
import torch
import pandas as pd
from scipy.stats import differential_entropy, norm
from itertools import product
import os
import logging
logging.getLogger().setLevel(logging.CRITICAL)
import warnings
warnings.filterwarnings('ignore')

# from google.colab import drive
# drive.mount('/content/drive')

# @title Runtime > Run All
class GPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


# Expected improvement, minimize y
def calculate_acf(pred_mean, pred_std, y_max):
    improve = y_max - pred_mean
    z_score = np.divide(improve, pred_std + 1e-9)
    acf = np.multiply(improve, norm.cdf(z_score)) + np.multiply(pred_std, norm.pdf(z_score))
    return max(acf,0)

def train_GP(train_x, train_y, train_iter):
    likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
    likelihood.noise = 10 ** -8
    model = GPModel(train_x, train_y, likelihood)
    model.train()
    likelihood.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in range(train_iter):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        optimizer.step()

    return model, likelihood

def predict_GP(GP_model, x):
  GP_model.eval()
  with torch.no_grad():
    f_preds = GP_model(x)
  return f_preds

def get_indices(index, dimensions):
    indices = []
    for dim_size in reversed(dimensions):
        index, remainder = divmod(index, dim_size)
        indices.insert(0, remainder)
    return tuple(indices)

In [ ]:
# @title 2. Add the path to the csv file

# Indicate the search space
sample_size = 100

# Indicate the file path
# To get the file path: right click on the file and click copy path
file_name = "PathToFile.csv"

assert os.path.exists(file_name)

In [ ]:
# @title 3. Indicate the size of the design space limits

x = pd.read_csv(file_name, dtype=np.float64, header=None).to_numpy()

n = x.shape[1]-1
print("Detected {} dimesional space".format(n))
limits = []

for i in range(n):
  l = float(input("Enter the lower limit for x{}: ".format(i+1)))
  u = float(input("Enter the upper limit for x{}: ".format(i+1)))
  limits.append([l, u])
  print()

space_size = tuple([sample_size] * n)
x_sample_space = np.empty(sample_size).T
for i in range(n):
  _ = np.linspace(limits[i][0], limits[i][1],sample_size)
  x_sample_space = np.vstack((x_sample_space, _))
x_sample_space = x_sample_space[1:]
dimensions = np.array([sample_size]*n)

In [ ]:
# @title 4. Run to perform GP BO
iterations = 1
for i in range(iterations):
  inx = torch.from_numpy(x[:, :-1])
  iny = torch.from_numpy(x[:, -1])
  GP_model, GP_likelihood = train_GP(inx, iny, train_iter=1000)

  acq_func = np.zeros(tuple([sample_size] * n))
  mean = np.zeros(tuple([sample_size] * n))
  variance = np.zeros(tuple([sample_size] * n))
  lower_variance = np.zeros(tuple([sample_size] * n))
  mesh = {}

  GP_model.eval()
  GP_likelihood.eval()

  for index, point in enumerate(product(*x_sample_space)):
      indices = get_indices(index, dimensions)
      mesh[indices] = point
      input_value = torch.from_numpy(np.asarray(point))[None,:]

      model_preds = predict_GP(GP_model, input_value)

      acq_func[indices] = calculate_acf(model_preds.mean.numpy(), torch.sqrt(model_preds.variance).numpy(), np.min(x[-1]))
      mean[indices] = model_preds.mean.numpy()[0]
      variance[indices] = 1.96 * np.sqrt(model_preds.variance.numpy())

  location = np.unravel_index(acq_func.argmax(), acq_func.shape)
  te = mesh[location]
  print("Analyze points: {}".format(te))


# Additional Helper Functions

In [ ]:
# @title 5. Plot 1D and 2D cases
if len(mean.shape) == 2:
    x, y = np.meshgrid(*x_sample_space)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(x, y, mean, facecolors=plt.cm.viridis(mean / np.max(mean)), alpha=0.5, linewidth=0)

    # # Plot upper variance
    # ax.plot_surface(x, y, mean + variance, facecolors=plt.cm.cividis(mean / np.max(mean)), alpha=0.5, linewidth=0)

    # # Plot lower variance
    # ax.plot_surface(x, y, mean - variance, facecolors=plt.cm.inferno(mean / np.max(mean)), alpha=0.5, linewidth=0)
    ax.scatter(x[:,0], x[:, 1], x[:, 2], c=x[:,2], cmap='viridis')
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')

    plt.show()

if len(mean.shape) == 1:
    fig, ax = plt.subplots()
    ax.scatter(x.T[0],x.T[1], label="3")
    ax.plot(x_sample_space.T, mean, label="4")
    ax.fill_between(x_sample_space[0].T, (mean-variance), (mean+variance),alpha=0.3)
    plt.show()
    plt.plot(x_sample_space[0].T, acq_func, label="2")
    plt.show()

In [ ]:
# @title 6. Query a specific point
p = []
for i in range(n):
  p.append(float(input("Query for x{}: ".format(i))))

p = np.array(p)
input_value = torch.from_numpy(np.asarray(p))[None,:]

model_preds = predict_GP(GP_model, input_value)
print("The mean is {} and the variance is {} at the point {}".format(model_preds.mean.numpy(), 1.96 * np.sqrt(model_preds.variance.numpy()), p))